In [1]:
!pip install mxnet # 코랩 환경이기 때문에 앞에 !를 붙여야 한다.
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

  Using cached transformers-3.0.2-py3-none-any.whl (769 kB)
  Using cached tokenizers-0.8.1rc1-cp38-cp38-manylinux1_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.12.1
    Uninstalling tokenizers-0.12.1:
      Successfully uninstalled tokenizers-0.12.1


ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: 'INSTALLER'
Consider using the `--user` option or check the permissions.



In [2]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-yvy2hmpv
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-yvy2hmpv
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit e1f2f37055e7460d8427f6912579c0162cb69831
  Preparing metadata (setup.py) ... done
  Using cached tokenizers-0.12.1-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (6.6 MB)
  Attempting uninstall: tokenizers


    Found existing installation: tokenizers 0.8.1rc1
    Uninstalling tokenizers-0.8.1rc1:
      Successfully uninstalled tokenizers-0.8.1rc1


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [4]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW # 인공지능 모델의 초기값 지정 함수를 아담으로 지정한다.
from transformers.optimization import get_cosine_schedule_with_warmup

In [5]:
import os

n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
    print(torch.cuda.get_device_name(i))

1
Tesla T4


In [6]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [7]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /home/lab09/.cache/kobert_v1.zip
using cached model. /home/lab09/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [8]:
import pandas as pd
naturalTraining_data = pd.read_csv('테스트.csv',encoding='utf-8',index_col=0)


In [9]:
def emote_change(x):
    if x==0:
        x='부정'
    elif x==1:
        x='긍정'
    else:
        x='중립'
    return x

In [10]:
data1=naturalTraining_data
data1['emote']=data1['emote'].apply(emote_change)
data1

,press,title,emote,date
0,조선일보,코로나 이겨내고 엄마는 ‘희망’을 낳았다,긍정,20210101
1,조선일보,"靑비서실장 유영민, 민정수석에 신현수",중립,20210101
2,조선일보,"이재명 18.2%, 이낙연 16.2%, 윤석열 15.1%..오차범위 내 3강 구도",중립,20210101
3,조선일보,"난 그래도 낙관한다, 눈에도 바람에도 지지않기를",긍정,20210101
4,조선일보,"유영민, 두달 전 비서실장 되나 묻자 “풀 뜯고 사는데 무슨...",중립,20210101
5,조선일보,"노영민 “대통령 제대로 보필 못해 죄송”', '김상조 사의 반려한 文대통령 “현안 ...",중립,20210101
6,조선일보,"與후보 적합도, 이재명·이낙연 順… 野선 윤석열·홍준표 順",중립,20210101
7,조선일보,"野후보 찍어 정권교체” 49.9%, “與후보 뽑아 정권유지",중립,20210101
8,조선일보,"4개월 의식불명 끝에 깨어난 美간호사…', '아가들아, 버텨줘서 고마워",긍정,20210101
9,조선일보,코로나 사망 하루 1000명… ‘방역 우등생’ 독일의 추락,부정,20210101


In [11]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(data1['emote'])
data1['emote'] = encoder.transform(data1['emote'])
data1.head(10)

,press,title,emote,date
0,조선일보,코로나 이겨내고 엄마는 ‘희망’을 낳았다,0,20210101
1,조선일보,"靑비서실장 유영민, 민정수석에 신현수",2,20210101
2,조선일보,"이재명 18.2%, 이낙연 16.2%, 윤석열 15.1%..오차범위 내 3강 구도",2,20210101
3,조선일보,"난 그래도 낙관한다, 눈에도 바람에도 지지않기를",0,20210101
4,조선일보,"유영민, 두달 전 비서실장 되나 묻자 “풀 뜯고 사는데 무슨...",2,20210101
5,조선일보,"노영민 “대통령 제대로 보필 못해 죄송”', '김상조 사의 반려한 文대통령 “현안 ...",2,20210101
6,조선일보,"與후보 적합도, 이재명·이낙연 順… 野선 윤석열·홍준표 順",2,20210101
7,조선일보,"野후보 찍어 정권교체” 49.9%, “與후보 뽑아 정권유지",2,20210101
8,조선일보,"4개월 의식불명 끝에 깨어난 美간호사…', '아가들아, 버텨줘서 고마워",0,20210101
9,조선일보,코로나 사망 하루 1000명… ‘방역 우등생’ 독일의 추락,1,20210101


In [12]:
mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
mapping

{0: '긍정', 1: '부정', 2: '중립'}

In [13]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data1, test_size=0.2, random_state=42) # new_df 오탈자 수정
print("train shape is:", len(train))
print("test shape is:", len(test))

train shape is: 8
test shape is: 3


In [14]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /home/lab09/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [15]:
test

,press,title,emote,date
5,조선일보,"노영민 “대통령 제대로 보필 못해 죄송”', '김상조 사의 반려한 文대통령 “현안 ...",2,20210101
0,조선일보,코로나 이겨내고 엄마는 ‘희망’을 낳았다,0,20210101
9,조선일보,코로나 사망 하루 1000명… ‘방역 우등생’ 독일의 추락,1,20210101


In [16]:
transform = nlp.data.BERTSentenceTransform(
            tok, max_seq_length=64, pad=True, pair=False) 
for i in range(len(test)):
    print(test.iloc[i][1])

노영민 “대통령 제대로 보필 못해 죄송”', '김상조 사의 반려한 文대통령 “현안 많아 교체할 때 아니다
코로나 이겨내고 엄마는 ‘희망’을 낳았다
코로나 사망 하루 1000명… ‘방역 우등생’ 독일의 추락


In [17]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 

        self.sentences = [transform([dataset.iloc[i][sent_idx]]) for i in range(len(dataset))]
        self.labels = [np.int32(dataset.iloc[i][label_idx]) for i in range(len(dataset))]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [18]:
# Setting parameters
max_len = 64 # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [19]:
data_train = BERTDataset(train, 1, 2, tok, max_len, True, False)
data_test = BERTDataset(test, 1, 2, tok, max_len, True, False)

In [20]:
# pytorch용 DataLoader 사용
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/home/ubuntu/anaconda3/envs/python3/lib/python3.8/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [21]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 3, # softmax 사용 <- binary일 경우는 2
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [22]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [23]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [24]:
# 옵티마이저 선언
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # softmax용 Loss Function 정하기 <- binary classification도 해당 loss function 사용 가능

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/home/ubuntu/anaconda3/envs/python3/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [25]:
# 학습 평가 지표인 accuracy 계산 -> 얼마나 타겟값을 많이 맞추었는가
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
  
# 모델 학습 시작
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm) # gradient clipping
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval() # 평가 모드로 변경
    
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/tmp/ipykernel_6094/1806958373.py:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/1 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.1809074878692627 train acc 0.25
epoch 1 train acc 0.25


/tmp/ipykernel_6094/1806958373.py:32: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/1 [00:00<?, ?it/s]

...

epoch 4 test acc 0.3333333333333333


  0%|          | 0/1 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.9980216026306152 train acc 0.875
epoch 5 train acc 0.875


  0%|          | 0/1 [00:00<?, ?it/s]

epoch 5 test acc 0.3333333333333333


In [48]:
# 테스트 문장 예측
test_sentence = '이재명 당선.'
test_label = 0 # 감성 라벨

unseen_test = pd.DataFrame([[test_sentence, test_label]], columns = [['뉴스 타이틀', '감성']])
unseen_values = unseen_test.values

test_set = BERTDataset(unseen_test, 0, 1, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=5)

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  out = model(token_ids, valid_length, segment_ids)
    
test_eval=[]
for i in out:
        logits=i
        logits = logits.detach().cpu().numpy()

        if np.argmax(logits) == 0:
            test_eval.append("부정")
        elif np.argmax(logits) == 1:
            test_eval.append("긍정")
        elif np.argmax(logits) == 2:
            test_eval.append("중립")

print(test_eval[0])

  

/tmp/ipykernel_6094/4193419368.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):


  0%|          | 0/1 [00:00<?, ?it/s]

중립
